In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import glob
from selenium import webdriver
import requests 
import os
import time
import numpy as np

In [6]:
receipt_data_file = glob.glob('C:/Users/hyb95_000/Desktop/체크카드 명세서/shinhancard_*.html') # 지정 폴더안에 있는 모든 파일을 불러내기 위함

total_df = []           # 일별 DataFrame 모음
total_df_region = []    # 지역별 DataFrame 모음
sheet_nm = []           # Excel Sheet Name 모음

# 위의 'total_df'와 'sheet_nm'에 데이터를 삽입하기 위한 for 구문
for url in receipt_data_file :
    driver = webdriver.Chrome('c:/chromedriver.exe')
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    title_tag = soup.select('head > title')
    title = title_tag[0].text

    # 암호된 문서일 경우 if 구문으로 들어감
    if title == '' :
        pw = '950730'
        driver.find_element_by_id('password').send_keys(pw)
        driver.find_element_by_xpath('//*[@id="aform"]/span/input').click()

    time.sleep(5)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    # 암호된 문서가 아닐 경우 아래있는 코드로 바로 들어감
    tables = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr')
    tables_middleSum = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr.middleSum')

    for n in range(len(tables_middleSum )) :
        tables.remove(tables_middleSum[n])
    
    # 명세서에서 추출한 데이터 string으로 변환하여 분리
    seperate_tables = []                          
    for n in range(len(tables)) :
        table = str(tables[n].text).split('\n')
        seperate_tables = seperate_tables + table

    # seperate_tables에서 '사용날짜' 추출   
    date_list = []
    for n in range(1,len(seperate_tables), 12) :
        date = seperate_tables[n]
        date_list.append(date)

    # seperate_tables에서 '사용처' 데이터만 추출    
    store_list = []
    for n in range(3,len(seperate_tables),12) :
        store = seperate_tables[n].replace(" ", "")
        store_list.append(store)

    # seperate_tables에서 추출된 '사용처' 데이터 --> 카카오 지도 API --> '사용처'에 대한 주소 추출      
    location_list = []
    def create_url(search_text):
        headers = {'Authorization': 'KakaoAK 3e8ea332f0cdef6293383ac7bf81a6fc'}
        base = "https://dapi.kakao.com/v2/local/search/keyword.json"
        param_query = "?query=" +search_text
    #     print(f'base + param_query: {base + param_query}')
        respond = requests.get(base + param_query,headers=headers)
        return respond.json()

    for n in range(len(store_list)) :
        store_name = store_list[n]
        result = create_url(store_name)
        tmp1 = result['documents']
        try :
            tmp2 = tmp1[0]
            tmp3 = tmp2['address_name']
            location_list.append(tmp3)
        except :
            tmp2 = np.nan
            location_list.append(tmp2)

    # seperate_tables에서 '사용금액' 데이터만 추출  
    price_list = [] 
    for n in range(4,len(seperate_tables),12) :
        price = seperate_tables[n]
        price_replace = int(price.replace(',',''))
        price_list.append(price_replace)
    
    # 추출한 데이터로 DataFrame 변환
    table_data = {'사용날짜':date_list, '사용처':store_list, '사용처주소':location_list,  '사용금액':price_list }
    
    df = pd.DataFrame(table_data, columns=['사용날짜', '사용처', '사용처주소', '사용금액'])
    df_sum = pd.DataFrame(df.sum(), columns=['합계'])
    df_total = df.append(df_sum.T)
    df_total.loc['합계', '사용날짜'] = '전체'
    df_total.loc['합계', '사용처'] = '전체'
    df_total.loc['합계', '사용처주소'] = '전체'
    total_df.append(df_total)
    
    ############################################################################# 여기서부터 지역별 데이터 파일데이터 추출
    
    df_dis_nan = df.dropna()

    df_address_list = list(df_dis_nan['사용처주소'])
    df_region_list = [ ' '.join(address.split()[:2]) for address in df_address_list]

    df_price_list = list(df_dis_nan['사용금액'])

    table_data_1 = {'사용지역':df_region_list, '사용금액':df_price_list}
    col_list = ['사용지역', '사용금액']
    df_Dup = pd.DataFrame(table_data_1 , columns = col_list ) # 중복값이 있는 df
    df_NoDup = df_Dup.drop_duplicates(subset = '사용지역')  # 중복값이 없는 df

    index_name_1 = df_Dup['사용지역']  # 중복값이 있는 사용지역 series
    df_Dup_name = df_Dup.rename(index_name_1)
    df_Dup_name = df_Dup_name.drop(['사용지역'], axis = 1)  # 중복값이 있고 사용지역이 index인 df

    index_name_2 = df_NoDup['사용지역']  # 중복값이 없는 사용지역 series
    df_NoDup_name = df_NoDup.rename(index_name_2)
    df_NoDup_name = df_NoDup_name.drop(['사용지역'], axis = 1)  # 중복값이 없고 사용지역이 index인 df

    for nm in list(index_name_2) :
        price_data = df_Dup_name.loc[nm]['사용금액']
        price_sum = price_data.sum()
        df_NoDup_name.loc[nm]['사용금액'] = price_sum

    name_region = df_Dup['사용지역']
    visit_counter = dict(name_region.value_counts())
    counter_list = []
    for k in index_name_2 :
        counter_list.append(visit_counter[k])
    
    df_NoDup_name.insert(0, '방문횟수', counter_list)
    total_df_region.append(df_NoDup_name)
    
    ############################################################################# 여기까지 지역별 데이터 파일데이터 추출  
    
    tables_date = soup.select('body > div.wrap.check > div.content > div.receiveDate > span.date')
    nm = str(tables_date[0].text)
    sheet_nm.append(nm)
    
    driver.close()

# 모아진 DataFrame을 Excel로변환
folder = 'C:/Users/hyb95_000/Desktop/체크카드 명세서'

if not os.path.exists(folder) :
    os.makedirs(folder)
    
excel_writer = pd.ExcelWriter('C:/Users/hyb95_000/Desktop/체크카드 명세서/CardUseList.xlsx', engine='xlsxwriter')
ToExcel_data =[]

for n in range(len(total_df)) :
    ToAppend_total_df = total_df[n].to_excel(excel_writer, index = True, sheet_name = sheet_nm[n]+'일별 데이터')
    ToExcel_data.append(ToAppend_total_df)
    
#     ToAppend_total_region_df = total_df_region[n].to_excel(excel_writer, index = True, sheet_name = sheet_nm[n]+'지역별 데이터')
    ToAppend_total_region_df = total_df_region[n].to_excel(excel_writer, index = True, sheet_name = sheet_nm[n]+'일별 데이터', startcol=6)
    ToExcel_data.append(ToAppend_total_region_df)

# for n in range(len(total_df)) :
#     ToAppend_df = total_df[n].to_excel(excel_writer, index = True, sheet_name = sheet_nm[n]+'일별 데이터')
#     ToExcel_data.append(ToAppend_df)
        
# for n in range(len(total_df_region)) :
#     ToAppend_df = total_df_region[n].to_excel(excel_writer, index = True, sheet_name = sheet_nm[n]+'지역별 데이터')
#     ToExcel_data.append(ToAppend_df)
        
excel_writer.save()

In [7]:
df_address = [] # 위치정보 없는 데이터 삭제한 DF 넣을 리스트
df_address_only = [] # 중복 사용지점 없는 DF 넣을 리스트
data_in_name = []
data_in_only_name = []
count_list = []

# 위치정보 없는 데이터 삭제한 DF 만들기
df_address = [df.dropna() for df in total_df]

# 중복 사용지점 없는 DF 만들기
df_address_only = [df.drop_duplicates(subset = '사용처') for df in df_address]

for i in range(len(df_address)):
    
    name_index =  df_address[i]['사용처']
    data_in_name.append(df_address[i].rename(name_index))
    data_in_name[i] = data_in_name[i].drop(['사용처'], axis = 1)

    name_index =  df_address_only[i]['사용처']
    data_in_only_name.append(df_address_only[i].rename(name_index))
    data_in_only_name[i] = data_in_only_name[i].drop(['사용처'], axis = 1)

#     for name in list(name_index):
#         tmp_data = data_in_name[i].loc[name]['사용금액']

#         if type(tmp_data) != str:
#             price = [price.replace(',','') for price in tmp_data]
#             price = list(map(int, price))
#             price_sum = sum(price)
#             data_in_only_name.loc[name]['사용금액'] = price_sum

#         else:
#             data_in_only_name.loc[name]['사용금액'] = int(tmp_data.replace(',',''))

    name_series = df_address[i]['사용처']

    visit_count = dict(name_series.value_counts())

    count_list_tmp = []

    for k in name_index:
        count_list_tmp.append(visit_count[k])

        
    count_list.append(count_list_tmp)
    
    # 사용횟수 열 추가하기

    data_in_only_name[i].insert(3, '방문횟수', count_list[i])
    

In [8]:
# date = list(data_in_only_name['사용날짜'])
# store_name = list(name_index)
# address = list(data_in_only_name['사용처주소'])
# price = list(data_in_only_name['사용금액'])
# count = count_list

In [28]:
import requests

# date = date_list
# store_name = store_list
# address = location_list
# price = price_list


# address = ['동작대로 29길 91', '서초구 효령로 335']
coordinates_x = []
coordinates_y = []
# store_name = ['우리집', '플레이데이터']

# price = ['10000', '2000']
 
def getLatLng(address):
    result = ""
 
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    rest_api_key = '289858dc68417f30677b064987b71bab'
    header = {'Authorization': 'KakaoAK ' + rest_api_key}
 
    r = requests.get(url, headers=header)
 
    if r.status_code == 200:
        try:
            result_address = r.json()["documents"][0]["address"]
            result = result_address["y"], result_address["x"]
        except:
            result = False
    else:
        result = "ERROR[" + str(r.status_code) + "]"
    
    return result
 
    
# 카카오 REST API로 좌표 구하기
# for j in address:
    
#     if j != False:
#         coordinates_x.append(getLatLng(j)[0])
#         coordinates_y.append(getLatLng(j)[1])
#     else:
#         coordinates_x.append('')
#         coordinates_y.append('')
        
    
# n = len(address)

# center_x_tmp = .0
# center_y_tmp = .0

# for k in range(len(store_name)):
#         center_x_tmp = center_x_tmp + float(coordinates_x[k])
#         center_y_tmp = center_y_tmp + float(coordinates_y[k])
    
# center_x = center_x_tmp / n
# center_y = center_y_tmp / n

In [29]:
for i in range(len(data_in_only_name)):

    date = list(data_in_only_name[i]['사용날짜'][:-1])
    store_name = list(data_in_only_name[i].index[:-1])
    address = list(data_in_only_name[i]['사용처주소'][:-1])
    price = list(data_in_only_name[i]['사용금액'][:-1])
    count = count_list[i][:-1]

    coordinates_x = []
    coordinates_y = []
    

    for j in address:
        
        coor = getLatLng(j)
        

        if coor != False:
            coordinates_x.append(coor[0])
            coordinates_y.append(coor[1])
        else:
            coordinates_x.append(False)
            coordinates_y.append(False)



    html = '<div id="map" style="width:100%;height:100%;"></div>'
    html = html + '<script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey=562beaefa8f3396ba21741328654bece"></script>\n'
    html = html + '<script>\n'
    html = html + "var mapContainer = document.getElementById('map'), \n"
    html = html + '    mapOption = { \n'

    html = html + '        center: new kakao.maps.LatLng(37.5642135, 127.0016985), \n'
    html = html + '        level: 7 \n'
    html = html + '    };'


    html = html + """
    var map = new kakao.maps.Map(mapContainer, mapOption); // 지도를 생성합니다

    // 마커를 표시할 위치와 title 객체 배열입니다 

    var positions = []
    """
    for k in range(len(store_name)):
        if coordinates_x[k] == False:
            continue
        html = html + "positions.push({ title: " + f"'{store_name[k]}', price: '{price[k]}', count: '{count[i]}'," + f" latlng: new kakao.maps.LatLng({coordinates_x[k]}, {coordinates_y[k]})" + "})\n"

    html = html + """
    // 마커 이미지의 이미지 주소입니다
    var imageSrc = "https://t1.daumcdn.net/localimg/localimages/07/mapapidoc/markerStar.png"; 

    for (var i = 0; i < positions.length; i ++) {

        // 마커 이미지의 이미지 크기 입니다
        var imageSize = new kakao.maps.Size(24, 35); 

        // 마커 이미지를 생성합니다    
        var markerImage = new kakao.maps.MarkerImage(imageSrc, imageSize); 

        // 마커를 생성합니다
        var marker = new kakao.maps.Marker({
            map: map, // 마커를 표시할 지도
            position: positions[i].latlng, // 마커를 표시할 위치
            title : positions[i].title, // 마커의 타이틀, 마커에 마우스를 올리면 타이틀이 표시됩니다
            image : markerImage // 마커 이미지 
        });

        var infowindow = new kakao.maps.InfoWindow({
            position : positions[i].latlng, 
            content : '<div style="width:100%;height:100%;font-size:11px">상호명 : ' + positions[i].title + '<br>방문횟수 : ' + positions[i].count + '<br>사용금액 : ' + positions[i].price +  '</div>'
        });


        infowindow.open(map, marker); 

    }
    </script>"""

    map_file = open('map' + str(i) + '.html', 'w')
    map_file.write(html)
    map_file.close()

    print(html)

<div id="map" style="width:100%;height:100%;"></div><script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey=562beaefa8f3396ba21741328654bece"></script>
<script>
var mapContainer = document.getElementById('map'), 
    mapOption = { 
        center: new kakao.maps.LatLng(37.5642135, 127.0016985), 
        level: 7 
    };
    var map = new kakao.maps.Map(mapContainer, mapOption); // 지도를 생성합니다

    // 마커를 표시할 위치와 title 객체 배열입니다 

    var positions = []
    positions.push({ title: '자판기', price: '1000', count: '1', latlng: new kakao.maps.LatLng(37.5383303723553, 126.987097452984)})
positions.push({ title: '다이소(공릉점)', price: '5000', count: '1', latlng: new kakao.maps.LatLng(37.6264557243819, 127.074491271089)})
positions.push({ title: '세븐일레븐노원공릉그린점', price: '2300', count: '1', latlng: new kakao.maps.LatLng(37.6245976355883, 127.0783613752)})
positions.push({ title: '이마트24노원공릉점', price: '2400', count: '1', latlng: new kakao.maps.LatLng(37.6249345631284, 127.077743114464)})


<div id="map" style="width:100%;height:100%;"></div><script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey=562beaefa8f3396ba21741328654bece"></script>
<script>
var mapContainer = document.getElementById('map'), 
    mapOption = { 
        center: new kakao.maps.LatLng(37.5642135, 127.0016985), 
        level: 7 
    };
    var map = new kakao.maps.Map(mapContainer, mapOption); // 지도를 생성합니다

    // 마커를 표시할 위치와 title 객체 배열입니다 

    var positions = []
    positions.push({ title: '자판기', price: '1000', count: '1', latlng: new kakao.maps.LatLng(37.5383303723553, 126.987097452984)})
positions.push({ title: '빽다방서울과기대점', price: '3500', count: '1', latlng: new kakao.maps.LatLng(37.6286765044367, 127.077111260585)})
positions.push({ title: '세븐일레븐노원공릉그린점', price: '1000', count: '1', latlng: new kakao.maps.LatLng(37.6245976355883, 127.0783613752)})
positions.push({ title: '빽다방안암역점', price: '2000', count: '1', latlng: new kakao.maps.LatLng(37.5866684958712, 127.030109549294)})
po

<div id="map" style="width:100%;height:100%;"></div><script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey=562beaefa8f3396ba21741328654bece"></script>
<script>
var mapContainer = document.getElementById('map'), 
    mapOption = { 
        center: new kakao.maps.LatLng(37.5642135, 127.0016985), 
        level: 7 
    };
    var map = new kakao.maps.Map(mapContainer, mapOption); // 지도를 생성합니다

    // 마커를 표시할 위치와 title 객체 배열입니다 

    var positions = []
    positions.push({ title: '악쓰는하마코인노래연습장2호점', price: '2000', count: '1', latlng: new kakao.maps.LatLng(37.6188160731152, 126.919604033116)})
positions.push({ title: '굿모닝24시', price: '9000', count: '1', latlng: new kakao.maps.LatLng(37.5576808976743, 126.955201479765)})
positions.push({ title: '쿠팡', price: '4890', count: '1', latlng: new kakao.maps.LatLng(37.5157751181814, 127.099062233457)})
positions.push({ title: '씨유이수나누리점', price: '3000', count: '1', latlng: new kakao.maps.LatLng(37.4876448251511, 126.981013458035)})


<div id="map" style="width:100%;height:100%;"></div><script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey=562beaefa8f3396ba21741328654bece"></script>
<script>
var mapContainer = document.getElementById('map'), 
    mapOption = { 
        center: new kakao.maps.LatLng(37.5642135, 127.0016985), 
        level: 7 
    };
    var map = new kakao.maps.Map(mapContainer, mapOption); // 지도를 생성합니다

    // 마커를 표시할 위치와 title 객체 배열입니다 

    var positions = []
    positions.push({ title: '메가커피이수태평점', price: '3900', count: '3', latlng: new kakao.maps.LatLng(37.4876352550898, 126.980893266007)})
positions.push({ title: '씨유서울과학기술대1관점', price: '4230', count: '3', latlng: new kakao.maps.LatLng(37.6345240613145, 127.07728112475)})
positions.push({ title: '유림복사', price: '2750', count: '3', latlng: new kakao.maps.LatLng(37.5834557806779, 126.998465360571)})
positions.push({ title: '맘스터치', price: '4000', count: '3', latlng: new kakao.maps.LatLng(37.5178815192915, 126.906167641009)})
pos

<div id="map" style="width:100%;height:100%;"></div><script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey=562beaefa8f3396ba21741328654bece"></script>
<script>
var mapContainer = document.getElementById('map'), 
    mapOption = { 
        center: new kakao.maps.LatLng(37.5642135, 127.0016985), 
        level: 7 
    };
    var map = new kakao.maps.Map(mapContainer, mapOption); // 지도를 생성합니다

    // 마커를 표시할 위치와 title 객체 배열입니다 

    var positions = []
    positions.push({ title: '써브웨이', price: '12800', count: '1', latlng: new kakao.maps.LatLng(35.1548850769753, 126.847780829911)})
positions.push({ title: '공차서울과학기술대점', price: '4500', count: '1', latlng: new kakao.maps.LatLng(37.6293964159781, 127.076521447735)})
positions.push({ title: '투썸플레이스대학로점', price: '3800', count: '1', latlng: new kakao.maps.LatLng(37.5791742549942, 127.001795612107)})
positions.push({ title: '주)스타벅스커피코리아', price: '4100', count: '1', latlng: new kakao.maps.LatLng(37.5601755439283, 126.98292106107